In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [2]:
file_fores=rf"C:\Users\Lilian\OneDrive - purdue.edu\VS code\Data\ATC\COMPARISON\Forecast_point_data.xlsx"

In [3]:
# Read data
ur_df = pd.read_excel('Forecast_point_data.xlsx', sheet_name='UR')
nop_df = pd.read_excel('Forecast_point_data.xlsx', sheet_name='NoP')

# Extract group (first letter after IN_)
ur_df['group'] = ur_df['unique_id'].str.extract(r'IN_([A-Z])')[0]
nop_df['group'] = nop_df['unique_id'].str.extract(r'IN_([A-Z])')[0]

# Create quarter label (YYYY-Q#)
ur_df['quarter'] = ur_df['ds'].dt.year.astype(str) + '-Q' + ur_df['ds'].dt.quarter.astype(str)
nop_df['quarter'] = nop_df['ds'].dt.year.astype(str) + '-Q' + nop_df['ds'].dt.quarter.astype(str)

# Define quarter order for proper sorting
quarter_order = ['2025-Q1', '2025-Q2', '2025-Q3', '2025-Q4', '2026-Q1', '2026-Q2', '2026-Q3', '2026-Q4']

# Aggregate by group and quarter ONLY (sum all unique_ids within each group per quarter)
ur_agg = ur_df.groupby(['group', 'quarter'])['Best model'].sum().reset_index()
nop_agg = nop_df.groupby(['group', 'quarter'])['Best model'].sum().reset_index()

# Convert quarter to categorical for proper ordering
ur_agg['quarter'] = pd.Categorical(ur_agg['quarter'], categories=quarter_order, ordered=True)
nop_agg['quarter'] = pd.Categorical(nop_agg['quarter'], categories=quarter_order, ordered=True)

ur_agg = ur_agg.sort_values(['group', 'quarter'])
nop_agg = nop_agg.sort_values(['group', 'quarter'])


def get_colors(n, cmap_name):
    """Generate n colors from a colormap, avoiding the lightest shades."""
    cmap = plt.colormaps.get_cmap(cmap_name)
    return [cmap(0.3 + 0.6 * i / (n - 1 if n > 1 else 1)) for i in range(n)]


In [4]:
fig1, ax1 = plt.subplots(figsize=(12, 7), facecolor='#f8f9fa')
ax1.set_facecolor('#fafafa')
groups_ur = sorted(ur_agg['group'].unique())
colors_ur = sns.color_palette("husl", len(groups_ur))

for i, group in enumerate(groups_ur):
    data = ur_agg[ur_agg['group'] == group].sort_values('quarter')
    ax1.plot(data['quarter'].astype(str), data['Best model']/1e6, marker='o', label=f'Group {group}', 
             color=colors_ur[i], linewidth=2.5, markersize=9, markeredgecolor='white', markeredgewidth=1.2)

ax1.set_title('UR: Aggregated by ATC-1', fontweight='bold', fontsize=14, color='#2c3e50', pad=15)
ax1.set_xlabel('Quarter', fontsize=12, color='#34495e')
ax1.set_ylabel('Units Reimbursed (Millions)', fontsize=12, color='#34495e')
ax1.tick_params(axis='x', rotation=45, labelsize=11)
ax1.tick_params(axis='y', labelsize=11)
ax1.grid(True, alpha=0.4, linestyle='--')
ax1.legend(loc='upper left', bbox_to_anchor=(1.02, 1), fontsize=10, framealpha=0.95, borderaxespad=0)
plt.tight_layout()
fig1.savefig('plot1_UR_aggregated.png', dpi=180, bbox_inches='tight', facecolor='#f8f9fa')
plt.close(fig1)

fig2, ax2 = plt.subplots(figsize=(12, 7), facecolor='#f8f9fa')
ax2.set_facecolor('#fafafa')
groups_nop = sorted(nop_agg['group'].unique())
colors_nop = sns.color_palette("husl", len(groups_nop))

for i, group in enumerate(groups_nop):
    data = nop_agg[nop_agg['group'] == group].sort_values('quarter')
    ax2.plot(data['quarter'].astype(str), data['Best model']/1e3, marker='s', label=f'Group {group}', 
             color=colors_nop[i], linewidth=2.5, markersize=9, markeredgecolor='white', markeredgewidth=1.2)

ax2.set_title('NoP: Aggregated by ATC-1 (Quarterly)', fontweight='bold', fontsize=14, color='#2c3e50', pad=15)
ax2.set_xlabel('Quarter', fontsize=12, color='#34495e')
ax2.set_ylabel('Number of Prescriptions (Thousands)', fontsize=12, color='#34495e')
ax2.tick_params(axis='x', rotation=45, labelsize=11)
ax2.tick_params(axis='y', labelsize=11)
ax2.grid(True, alpha=0.4, linestyle='--')
ax2.legend(loc='upper left', bbox_to_anchor=(1.02, 1), fontsize=10, framealpha=0.95, borderaxespad=0)
plt.tight_layout()
fig2.savefig('plot2_NoP_aggregated.png', dpi=180, bbox_inches='tight', facecolor='#f8f9fa')
plt.close(fig2)

fig3, ax3 = plt.subplots(figsize=(12, 7), facecolor='#f8f9fa')
ax3.set_facecolor('#fafafa')

# Sort individual data by quarter
ur_df['quarter'] = pd.Categorical(ur_df['quarter'], categories=quarter_order, ordered=True)
ur_df = ur_df.sort_values(['unique_id', 'quarter'])

ur_ids = ur_df['unique_id'].unique()[:10]
colors_ur_ind = sns.color_palette("husl", len(ur_ids))

for i, uid in enumerate(ur_ids):
    data = ur_df[ur_df['unique_id'] == uid].sort_values('quarter')
    ax3.plot(data['quarter'].astype(str), data['Best model']/1e6, marker='D', label=uid, 
             color=colors_ur_ind[i], linewidth=2.5, markersize=8, markeredgecolor='white', markeredgewidth=1)

ax3.set_title('UR: Individual Unique IDs (First 10)', fontweight='bold', fontsize=14, color='#2c3e50', pad=15)
ax3.set_xlabel('Quarter', fontsize=12, color='#34495e')
ax3.set_ylabel('Units Reimbursed (Millions)', fontsize=12, color='#34495e')
ax3.tick_params(axis='x', rotation=45, labelsize=11)
ax3.tick_params(axis='y', labelsize=11)
ax3.grid(True, alpha=0.4, linestyle='--')
ax3.legend(loc='upper left', bbox_to_anchor=(1.02, 1), fontsize=10, framealpha=0.95, borderaxespad=0)
plt.tight_layout()
fig3.savefig('plot3_UR_individual.png', dpi=180, bbox_inches='tight', facecolor='#f8f9fa')
plt.close(fig3)

fig4, ax4 = plt.subplots(figsize=(12, 7), facecolor='#f8f9fa')
ax4.set_facecolor('#fafafa')

# Sort individual data by quarter
nop_df['quarter'] = pd.Categorical(nop_df['quarter'], categories=quarter_order, ordered=True)
nop_df = nop_df.sort_values(['unique_id', 'quarter'])

nop_ids = nop_df['unique_id'].unique()[:10]
colors_nop_ind = sns.color_palette("husl", len(nop_ids))

for i, uid in enumerate(nop_ids):
    data = nop_df[nop_df['unique_id'] == uid].sort_values('quarter')
    ax4.plot(data['quarter'].astype(str), data['Best model']/1e3, marker='^', label=uid, 
             color=colors_nop_ind[i], linewidth=2.5, markersize=8, markeredgecolor='white', markeredgewidth=1)

ax4.set_title('NoP: Individual Unique IDs (First 10)', fontweight='bold', fontsize=14, color='#2c3e50', pad=15)
ax4.set_xlabel('Quarter', fontsize=12, color='#34495e')
ax4.set_ylabel('Number of Prescriptions (Thousands)', fontsize=12, color='#34495e')
ax4.tick_params(axis='x', rotation=45, labelsize=11)
ax4.tick_params(axis='y', labelsize=11)
ax4.grid(True, alpha=0.4, linestyle='--')
ax4.legend(loc='upper left', bbox_to_anchor=(1.02, 1), fontsize=10, framealpha=0.95, borderaxespad=0)
plt.tight_layout()
fig4.savefig('plot4_NoP_individual.png', dpi=180, bbox_inches='tight', facecolor='#f8f9fa')
plt.close(fig4)

print("All 4 plots saved successfully!")

All 4 plots saved successfully!
